In [0]:
import torch as tr
#import numpy as np
import pandas
#CHAR_CHANNELS = 55

class CharTokenizer():
    def __init__(self):
        #emb = np.eye(80, 55)
        #emb[54:80] = emb[28:54]
        #emb[28:54, 54] = 1
        emb = tr.eye(80,80)
        self.embedding = emb

    def tokenize(self, comments): #comments es lista de palabras
        return list(map(lambda x:self.str2tns(x), comments)) #x->a cada elemento de comments
                                                            #(palabra) le aplico str2tns

    def str2tns(self, comm): #comm es una palabra
        return tr.LongTensor(list(map(self.char2idx, comm))) #a cada elemento de la palabra 
                                                #(caracter) le aplico char2idx

    def char2idx(self, c): #c es un caracter
        idx = 27     
        #print(c)
        if c in self.char_set:
            idx = self.char_set[c]
        return idx
    
    #def get_tensor(self,idx):
    #    return list(map(lambda x: self.embedding[x],idx))

    char_set = {
            ' ' : 0, '\n': 1, '!' :  2, '"' :  3, '#' :  4, '$' :  5, '%' :  6, '&' :  7, "'" :  8,
            '(' :  9, ')' : 10, '*' : 11, '+' : 12, ',' : 13, '-' : 14, '.' : 15, '/' : 16, ':' : 17,
            ';' : 18, '=' : 19, '?' : 20, '_' : 21, '\xad' : 22, '’' : 23, '“' : 24, '”' : 25,
            '0' : 26, '1' : 26, '2' : 26, '3' : 26, '4' : 26, '5' : 26, '6' : 26, '7' : 26, '8' : 26,
            '9' : 26,
            'A' : 28, 'B' : 29, 'C' : 30, 'D' : 31, 'E' : 32, 'F' : 33, 'G' : 34, 'H' : 35, 'I' : 36,
            'J' : 37, 'K' : 38, 'L' : 39, 'M' : 40, 'N' : 41, 'O' : 42, 'P' : 43, 'Q' : 44, 'R' : 45,
            'S' : 46, 'T' : 47, 'U' : 48, 'V' : 49, 'W' : 50, 'X' : 51, 'Y' : 52, 'Z' : 53,
            'a' : 28, 'b' : 29, 'c' : 30, 'd' : 31, 'e' : 32, 'f' : 33, 'g' : 34, 'h' : 35, 'i' : 36,
            'j' : 37, 'k' : 38, 'l' : 39, 'm' : 40, 'n' : 41, 'o' : 42, 'p' : 43, 'q' : 44, 'r' : 45,
            's' : 46, 't' : 47, 'u' : 48, 'v' : 49, 'w' : 50, 'x' : 51, 'y' : 52, 'z' : 53}
    """
    char_set={'a':0,'b':1,'c':2}
    """


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import torch as torch
from torch import nn
import numpy as np
import pandas as pn
#from char_tokenizer import CharTokenizer
#import utils
from sklearn.metrics import balanced_accuracy_score
import torch.nn.functional as F
#from utils import EarlyStopping
from matplotlib import pyplot as plt
!pip install skorch
import skorch
from skorch import NeuralNetClassifier,NeuralNet
from sklearn.model_selection import RandomizedSearchCV,train_test_split,GridSearchCV
from mpl_toolkits import mplot3d
import scipy.interpolate as interp
from mpl_toolkits.mplot3d import Axes3D


In [0]:
def one_hot_encode(indx, dict_size, maxlen, batch_size):
    # Creating a multi-dimensional array of zeros with the desired output shape
    features = np.zeros((batch_size, maxlen, dict_size), dtype=np.float32)
    
    # Replacing the 0 at the relevant character index with a 1 to represent that character
    for i in range(batch_size):
        for u in range(maxlen):
            features[i, u, int(indx[i][u])] = 1
    return features

In [0]:
class CNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size,maxlen,num_layers):
        super(CNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers

        if num_layers == 2:
            self.block=nn.Sequential(nn.Conv1d(input_size,hidden_size,3,padding=1),nn.ReLU(),nn.BatchNorm1d(hidden_size),nn.AvgPool1d(2),
            nn.Conv1d(hidden_size,hidden_size,3,padding=1),nn.ReLU(),nn.BatchNorm1d(hidden_size),nn.AvgPool1d(2))
        elif num_layers == 3:
            self.block=nn.Sequential(nn.Conv1d(input_size,hidden_size,3,padding=1),nn.ReLU(),nn.BatchNorm1d(hidden_size),nn.AvgPool1d(2),
            nn.Conv1d(hidden_size,hidden_size,3,padding=1),nn.ReLU(),nn.BatchNorm1d(hidden_size),nn.AvgPool1d(2),
            nn.Conv1d(hidden_size,hidden_size,3,padding=1),nn.ReLU(),nn.BatchNorm1d(hidden_size),nn.AvgPool1d(2))
        elif num_layers == 4:
            self.block=nn.Sequential(nn.Conv1d(input_size,hidden_size,3,padding=1),nn.ReLU(),nn.BatchNorm1d(hidden_size),nn.AvgPool1d(2),
            nn.Conv1d(hidden_size,hidden_size,3,padding=1),nn.ReLU(),nn.BatchNorm1d(hidden_size),nn.AvgPool1d(2),
            nn.Conv1d(hidden_size,hidden_size,3,padding=1),nn.ReLU(),nn.BatchNorm1d(hidden_size),nn.AvgPool1d(2),
            nn.Conv1d(hidden_size,hidden_size,3,padding=1),nn.ReLU(),nn.BatchNorm1d(hidden_size),nn.AvgPool1d(2))
        self.out = nn.Linear(hidden_size*int(maxlen/2**num_layers),output_size)
        """        
        self.c1 = nn.Conv1d(input_size, hidden_size, 3,padding=1)
        self.a1 = nn.ReLU()
        self.b1 = nn.BatchNorm1d(hidden_size)
        self.p1 = nn.AvgPool1d(2)
        
        self.c2 = nn.Conv1d(hidden_size, hidden_size, 3,padding=1)
        self.a2 = nn.ReLU()
        self.b2 = nn.BatchNorm1d(hidden_size)
        self.p2 = nn.AvgPool1d(2)
        
        self.c3 = nn.Conv1d(hidden_size, hidden_size, 3,padding=1)
        self.a3 = nn.ReLU()
        self.b3 = nn.BatchNorm1d(hidden_size)
        self.p3 = nn.AvgPool1d(2)
        
        #nn.Sequential(nn.Conv1d(input_size,hidden_size,3),
        #)
        #self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=0.01)
        #self.gru = nn.GRU(input_size,hidden_size,n_layers,dropout=0.01)
        self.out = nn.Linear(hidden_size*int(maxlen/2**num_layers), output_size)
        """

    def forward(self, inputs, hidden=None):
        #batch_size = inputs.size(1)
        #batch_size = inputs.size(0)
        #print("dimensiones del tensor que entra a la cnn: ",inputs.shape)
        # Turn (seq_len x batch_size x input_size) into (batch_size x input_size x seq_len) for CNN
        #inputs = inputs.transpose(0, 1).transpose(1, 2)

        #entra un tensor de dimensiones: batch x seq_len x input_size
        #quiero un tensor dedimensiones: batch x input_size x seq_len
        inputs = inputs.transpose(1, 2)
        x = self.block(inputs)
        #print(x.shape)
        """
        # Run through Conv1d and Pool1d layers
        c = self.c1(inputs)
        #print("Paso primera capa")
        #usar relu y batch norm a la salida 
        a = self.a1(c)
        b = self.b1(a)
        p = self.p1(b)
        #print("Paso segunda capa")
        c = self.c2(p)
        a = self.a2(c)
        b = self.b2(a)
        p = self.p2(b)
        #print("Paso tercera capa")
        #print("Paso cuarta capa")
        c = self.c3(p)
        a = self.a3(c)
        b = self.b3(a)
        p = self.p3(b)
        #print("dimensiones del tensor que sale de la cnn antes de transpose: ",p.shape)
        
        # Turn (batch_size x hidden_size x seq_len) back into (seq_len x batch_size x hidden_size) for RNN
        #p = p.transpose(1, 2).transpose(0, 1)
        #print("dimensiones del tensor que entra a la gru antes de transpose: ",p.shape)
        #sale tensor de batch_size x hidden_size x seq_len
        
        #quiero tensor de seq_len x batch_size x hidden_size
        #p = p.transpose(0,1).transpose(0,2)
        #print("dimensiones del tensor que entra a la gru dsp de transpose: ",p.shape)
        #p = torch.tanh(p)
        #output, hidden = self.gru(p, hidden)
        #print("Paso quinta capa")
        #print("dimension de lo que sale de la gru ",output.shape)
        
        #conv_seq_len = p.shape[2]
        #p = p.view(-1,conv_seq_len * self.hidden_size) # Treating (conv_seq_len x batch_size) as batch_size for linear layer
        """
        p = torch.flatten(x,start_dim=1)
        #output = torch.tanh(self.out(output))
        #output = self.out(output.reshape(-1,conv_seq_len*self.hidden_size))
        #output = self.out(output.reshape(conv_seq_len*batch_))
        output = self.out(p)
        #print("Paso ultima capa")
        #print("shape de la salida:",output.shape)
        
        #output = output.view(conv_seq_len, -1, self.output_size)
        return output

In [0]:
correctedData = pn.read_csv("preprocessedQuestions_lem.csv",delimiter=',',header=None) #comentar esta linea en caso de descomentar la anterior
cantidad_preg = correctedData.shape[0]
#Xtrain,Ytrain,Xtest,Ytest,Xval,Yval = utils.separate_dataset(correctedData.values,cantidad_preg,True)
print("llego hasta dsp de separate dataset")
#print(correctedData.values[:,1])
text = correctedData.values[:,1]
#labels = correctedData.values[:,0]
#labels = np.array(labels,dtype=np.int8)
# Finding the length of the longest string in our data
maxlen = len(max(text, key=len))
#print(maxlen)
# Padding

llego hasta dsp de separate dataset


In [0]:
# A simple loop that loops through the list of sentences and adds a ' ' whitespace until the length of
# the sentence matches the length of the longest sentence

for i in range(len(text)):
    while(len(text[i]))<maxlen:
        text[i] += ' '

#for i in range(len(Xtrain)):
#    while len(Xtrain[i,0])<maxlen:
#        Xtrain[i,0] += ' ' 

#for i in range(len(Xtest)):
#    while len(Xtest[i,0])<maxlen:
#        Xtest[i,0] += ' '

#for i in range(len(Xval)):
#    while len(Xval[i,0])<maxlen:
#        Xval[i,0] += ' '

print("llego hasta dsp de llenar con whitespace")

llego hasta dsp de llenar con whitespace


In [0]:
char = CharTokenizer()
indxChar = np.zeros((len(text),maxlen))
#indxTrain= np.zeros((len(Xtrain),maxlen))
#indxTest = np.zeros((len(Xtest),maxlen))
#indxVal = np.zeros((len(Xval),maxlen))
#indx es una matriz que tiene en cada fila los indices de los caracteres de la oracion, y maxlen columnas
for i in range(len(text)):
    indxChar[i] = char.tokenize(text[i])
#for i in range(len(Xtrain)):
#    indxTrain[i] = char.tokenize(Xtrain[i,0])
#for i in range(len(Xtest)):
#    indxTest[i] = char.tokenize(Xtest[i,0])
#for i in range(len(Xval)):
#    indxVal[i] = char.tokenize(Xval[i,0])
#print(indxTest[0])
#print(indxTest[60])

print("llego hasta dsp de char tokenize")

llego hasta dsp de char tokenize


In [0]:
dict_size = len(char.char_set)
#dict_size = 80
#print(dict_size)
#batch_size_train = len(Xtrain)
#batch_size_test = len(Xtest)
#batch_size_val = len(Xval)
#seq_len = maxlen
cant_preg = len(text)
X = one_hot_encode(indxChar,dict_size,maxlen,cant_preg)
Y = [x[0] for x in correctedData.values]
#input_seq_train = one_hot_encode(indxTrain,dict_size,maxlen,batch_size_train)
#input_seq_test = one_hot_encode(indxTest,dict_size,maxlen,batch_size_test)
#input_seq_val = one_hot_encode(indxVal,dict_size,maxlen,batch_size_val)
#input_seq_train = torch.from_numpy(input_seq_train)
#input_seq_test = torch.from_numpy(input_seq_test)
#input_seq_val = torch.from_numpy(input_seq_val)
#target_seq_train = Ytrain
#target_seq_test = Ytest
#target_seq_val = Yval
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

cantidad_labels = correctedData.values[len(correctedData.values)-1,0] + 1
# Instantiate the model with hyperparameters
#model = sRNN(input_size=dict_size, output_size=cantidad_labels, hidden_dim=12, n_layers=1)
#model = sRNN(input_size=dict_size, output_size=cantidad_labels, hidden_dim=20, n_layers=1,bidirectional=True)
# Define hyperparameters
n_epochs = 500
batch_size = 500
hidden_size = 20
num_layers=3
model = CNN(input_size=dict_size,hidden_size = hidden_size,output_size=cantidad_labels,maxlen=maxlen,num_layers=num_layers)
# We'll also set the model to the device that we defined earlier (default is CPU)
model = model.to(device)

parameters = {    
    #'module__hidden_dim' : [12,24,48,96],
    'module__hidden_size' : [20,50,80,40],    
    #'max_epochs' : [50,70,90,110,130,150]
    'module__num_layers':[2,3,4],
    'max_epochs' : [50,100],
    'batch_size' : [1000,500,700,150]
}




GPU is available


In [0]:
net = NeuralNetClassifier(model,module__input_size = dict_size,module__hidden_size=hidden_size,module__output_size=cantidad_labels,module__maxlen=maxlen,module__num_layers=num_layers,criterion=torch.nn.CrossEntropyLoss,optimizer=torch.optim.Adam,verbose=1,device=device)
gs = RandomizedSearchCV(net,parameters,verbose=2,n_jobs=-1,cv=10,scoring='balanced_accuracy',n_iter=2)
X_train,X_test,y_train,y_test = train_test_split(X,Y,shuffle=True,stratify=Y,test_size=0.1,random_state=12)
#y_train_tensor = torch.LongTensor(y_train)
#y_test_tensor = torch.LongTensor(y_test)
Y_numpy = np.asarray(y_train)

gs.fit(X_train,Y_numpy)

Fitting 10 folds for each of 2 candidates, totalling 20 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 9 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Re-initializing module because the following parameters were re-set: hidden_size, input_size, maxlen, num_layers, output_size.
Re-initializing module because the following parameters were re-set: hidden_size, input_size, maxlen, num_layers, output_size.
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        5.0461       0.0127        4.7539  0.1155


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  3.0min finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


      2        5.2540       0.0063        6.6060  0.1134
      3        4.4194       0.0095        5.4357  0.1097
      4        3.8710       0.0063        7.0854  0.1087
      5        3.4435       0.0158        6.6281  0.1119
      6        3.0267       0.0253        5.2838  0.1123
      7        2.6431       0.0348        5.3516  0.1110
      8        2.2461       0.0601        4.9793  0.1211
      9        1.8722       0.0506        5.8789  0.1138
     10        1.5684       0.1297        3.7297  0.1125
     11        1.2836       0.1171        5.3645  0.1121
     12        1.1266       0.2943        3.5244  0.1120
     13        0.9380       0.2848        3.8615  0.1130
     14        0.6867       0.2690        4.1197  0.1142
     15        0.4674       0.3481        3.3362  0.1134
     16        0.3229       0.4557        2.9736  0.1132
     17        0.2173       0.4304        2.8854  0.1174
     18        0.1419       0.3671        3.3115  0.1121
     19        0.0968       0.4

RandomizedSearchCV(cv=10, error_score='raise-deprecating',
                   estimator=<class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=CNN(
    (block): Sequential(
      (0): Conv1d(88, 20, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): ReLU()
      (2): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): AvgPool1d(kernel_size=(2,), stride=(2,),...
  module__maxlen=169,
  module__num_layers=3,
  module__output_size=106,
),
                   iid='warn', n_iter=2, n_jobs=-1,
                   param_distributions={'batch_size': [1000, 500, 700, 150],
                                        'max_epochs': [50, 100],
                                        'module__hidden_size': [20, 50, 80, 40],
                                        'module__num_layers': [2, 3, 4]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring='balanced_accuracy',


In [0]:
hidden_size = []
max_epochs = []
batch_size = []
num_layers = []
score = []
std=[]

# Utility function to report best scores (found online)
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        print(i)
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
            batch_size.append(results['params'][candidate]['batch_size'])
            hidden_size.append(results['params'][candidate]['module__hidden_size'])
            num_layers.append(results['params'][candidate]['module__num_layers'])
            max_epochs.append(results['params'][candidate]['max_epochs'])
            score.append(results['mean_test_score'][candidate])
            std.append(results['std_test_score'][candidate])
report(gs.cv_results_,2)


print("Batch size: ",batch_size)
print("Hidden Dim: ",hidden_size)
print("Max epochs: ",max_epochs)
print("Num layers: ",num_layers)
print("Score: ",score)
print("std: ",std)

1
Model with rank: 1
Mean validation score: 0.452 (std: 0.031)
Parameters: {'module__num_layers': 4, 'module__hidden_size': 50, 'max_epochs': 100, 'batch_size': 150}

2
Model with rank: 2
Mean validation score: 0.308 (std: 0.047)
Parameters: {'module__num_layers': 2, 'module__hidden_size': 80, 'max_epochs': 100, 'batch_size': 700}

Batch size:  [150, 700]
Hidden Dim:  [50, 80]
Max epochs:  [100, 100]
Num layers:  [4, 2]
Score:  [0.4518280946719958, 0.3077473223779089]
std:  [0.03140069462713873, 0.04719964138238955]


In [0]:
hidden_dim = hidden_size
ejex = hidden_dim
ejey = num_layers
ejez = score

plotx,ploty, = np.meshgrid(np.linspace(np.min(ejex),np.max(ejex),10),\
                           np.linspace(np.min(ejey),np.max(ejey),10))
plotz = interp.griddata((ejex,ejey),ejez,(plotx,ploty),method='cubic')
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.view_init(35,40)
ax.set_xlabel('Neuronas')
ax.set_ylabel('Capas')
ax.set_zlabel('Precisión')
ax.set_title('Precisión en función de la arquitectura')

surf = ax.plot_surface(plotx,ploty,plotz,vmin=np.nanmin(ejez),vmax = np.nanmax(ejez),cstride=1,rstride=1,cmap='viridis')
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.savefig("prueba4_Neuronas_Capas_Precision.svg")
#/////////////////////////////
#////////////////////////////

ejex = hidden_dim
ejey = num_layers
ejez = std

plotx,ploty, = np.meshgrid(np.linspace(np.min(ejex),np.max(ejex),10),\
                           np.linspace(np.min(ejey),np.max(ejey),10))
plotz = interp.griddata((ejex,ejey),ejez,(plotx,ploty),method='cubic')
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.view_init(35,40)
ax.set_xlabel('Neuronas')
ax.set_ylabel('Capas')
ax.set_zlabel('Std')
ax.set_title('Std en función de la arquitectura')

surf = ax.plot_surface(plotx,ploty,plotz,vmin=np.nanmin(ejez),vmax = np.nanmax(ejez),cstride=1,rstride=1,cmap='viridis')
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.savefig("prueba4_Neuronas_Capas_STD.svg")
#/////////////////////////////
#////////////////////////////

ejex = max_epochs
ejey = batch_size
ejez = score

plotx,ploty, = np.meshgrid(np.linspace(np.min(ejex),np.max(ejex),10),\
                           np.linspace(np.min(ejey),np.max(ejey),10))
plotz = interp.griddata((ejex,ejey),ejez,(plotx,ploty),method='cubic')
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.view_init(35,120)
ax.set_xlabel('Epocas')
ax.set_ylabel('Batch')
ax.set_zlabel('Precisión')
ax.set_title('Precisión en función de tamaño de batch y épocas')

surf = ax.plot_surface(plotx,ploty,plotz,vmin=np.nanmin(ejez),vmax = np.nanmax(ejez),cstride=1,rstride=1,cmap='viridis')
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.savefig("prueba4_Epocas_Batch_Precision.svg")

#/////////////////////////////
#////////////////////////////

ejex = max_epochs
ejey = batch_size
ejez = std

plotx,ploty, = np.meshgrid(np.linspace(np.min(ejex),np.max(ejex),10),\
                           np.linspace(np.min(ejey),np.max(ejey),10))
plotz = interp.griddata((ejex,ejey),ejez,(plotx,ploty),method='cubic')
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.view_init(35,120)
ax.set_xlabel('Epocas')
ax.set_ylabel('Batch')
ax.set_zlabel('Std')
ax.set_title('Std en función de tamaño de batch y épocas')

surf = ax.plot_surface(plotx,ploty,plotz,vmin=np.nanmin(ejez),vmax = np.nanmax(ejez),cstride=1,rstride=1,cmap='viridis')
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.savefig("prueba4_Epocas_Batch_STD.svg")

#/////////////////////////////
#////////////////////////////

ejex = hidden_dim
ejey = max_epochs
ejez = score

plotx,ploty, = np.meshgrid(np.linspace(np.min(ejex),np.max(ejex),10),\
                           np.linspace(np.min(ejey),np.max(ejey),10))
plotz = interp.griddata((ejex,ejey),ejez,(plotx,ploty),method='cubic')
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.view_init(35,120)
ax.set_xlabel('Neuronas')
ax.set_ylabel('Epocas')
ax.set_zlabel('Precisión')
ax.set_title('Precisión en función de neuronas y épocas')

surf = ax.plot_surface(plotx,ploty,plotz,vmin=np.nanmin(ejez),vmax = np.nanmax(ejez),cstride=1,rstride=1,cmap='viridis')
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.savefig("prueba4_Neuronas_Epocas_Precision.svg")

#print(X_test.shape)            
probs = gs.best_estimator_.predict_proba(X_test)
#print(probs.shape)

# get training and validation loss
epochs = [i for i in range(len(gs.best_estimator_.history))]
train_loss = gs.best_estimator_.history[:,'train_loss']
valid_loss = gs.best_estimator_.history[:,'valid_loss']
acc = balanced_accuracy_score(y_test_tensor,np.argmax(probs,axis=1))
print("tasa de acierto obtenida: ",acc)
fig1 = plt.figure()
plt.plot(epochs,train_loss,'g-')
plt.plot(epochs,valid_loss,'r-')
plt.title('Curvas del error en el entrenamiento')
plt.xlabel('Epocas')
plt.ylabel('Error (Cross Entropy Loss)')
plt.legend(['Entrenamiento','Validación'])
plt.savefig("prueba4_Loss.svg")
plt.show()

QhullError: ignored

In [0]:
acc = balanced_accuracy_score(y_test_tensor,np.argmax(probs,axis=1))
print("tasa de acierto obtenida: ",acc)

tasa de acierto obtenida:  0.5440251572327044
